In [118]:
import os
import time
import timeit
import redis
from tqdm import tqdm
from faker import Faker

In [119]:
master_port = 6379
branch_port = 6378

In [120]:
prefix = "/home/ubuntu/code/"
workdir = "RediSearchClean"  # Master build

# Remove dump file if it exists
if os.path.exists(f"{os.getcwd()}/dump.rdb"):
    os.remove(f"{os.getcwd()}/dump.rdb")

# Start Redis server with master build
artifact = f"{prefix}{workdir}/bin/linux-x64-release/search-community/redisearch.so"
os.system(f"redis-server --loadmodule {artifact} --port {master_port}&")
time.sleep(5)  # Give Redis some time to start up

# Load data into Redis
master_redis_client = redis.Redis(host='localhost', port=master_port)
master_redis_client.execute_command("FLUSHALL")

233280:C 09 Apr 2025 15:59:46.433 # WARNING Memory overcommit must be enabled! Without it, a background save or replication may fail under low memory condition. Being disabled, it can also cause failures without low memory condition, see https://github.com/jemalloc/jemalloc/issues/1328. To fix this issue add 'vm.overcommit_memory = 1' to /etc/sysctl.conf and then reboot or run the command 'sysctl vm.overcommit_memory=1' for this to take effect.
233280:C 09 Apr 2025 15:59:46.433 * oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
233280:C 09 Apr 2025 15:59:46.433 * Redis version=255.255.255, bits=64, commit=3cdb8c60, modified=0, pid=233280, just started
233280:C 09 Apr 2025 15:59:46.433 * Configuration loaded
233280:M 09 Apr 2025 15:59:46.433 * monotonic clock: POSIX clock_gettime
233280:M 09 Apr 2025 15:59:46.434 * Running mode=standalone, port=6379.
233280:M 09 Apr 2025 15:59:46.436 * <search> Redis version found by RedisSearch : 255.255.255 - oss
233280:M 09 Apr 2025 15:59:46.436 * <sear

True

In [121]:
prefix = "/home/ubuntu/code/"
workdir = "RediSearch"  # branch build

# Remove dump file if it exists
if os.path.exists(f"{os.getcwd()}/dump.rdb"):
    os.remove(f"{os.getcwd()}/dump.rdb")

# Start Redis server with master build
artifact = f"{prefix}{workdir}/bin/linux-x64-release/search-community/redisearch.so"
os.system(f"redis-server --loadmodule {artifact} --port {branch_port} &")
time.sleep(5)  # Give Redis some time to start up

# Load data into Redis
branch_redis_client = redis.Redis(host='localhost', port=branch_port)
branch_redis_client.execute_command("FLUSHALL")

233300:C 09 Apr 2025 15:59:52.381 # WARNING Memory overcommit must be enabled! Without it, a background save or replication may fail under low memory condition. Being disabled, it can also cause failures without low memory condition, see https://github.com/jemalloc/jemalloc/issues/1328. To fix this issue add 'vm.overcommit_memory = 1' to /etc/sysctl.conf and then reboot or run the command 'sysctl vm.overcommit_memory=1' for this to take effect.
233300:C 09 Apr 2025 15:59:52.381 * oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
233300:C 09 Apr 2025 15:59:52.381 * Redis version=255.255.255, bits=64, commit=3cdb8c60, modified=0, pid=233300, just started
233300:C 09 Apr 2025 15:59:52.381 * Configuration loaded
233300:M 09 Apr 2025 15:59:52.381 * monotonic clock: POSIX clock_gettime
233300:M 09 Apr 2025 15:59:52.382 * Running mode=standalone, port=6378.
233300:M 09 Apr 2025 15:59:52.383 * <search> Redis version found by RedisSearch : 255.255.255 - oss
233300:M 09 Apr 2025 15:59:52.383 * <sear

True

In [123]:
clients = [branch_redis_client, master_redis_client]

In [146]:

# Create RediSearch index - updated to reflect that CID is used in doc name, not as a field
index_cmd = f"""
FT.CREATE idx ON HASH PREFIX 1 doc:
SCHEMA
    t TEXT
"""
for client in clients:
    client.execute_command(index_cmd)
    print(f"Created search index: idx")

Created search index: idx
Created search index: idx


In [125]:
def populate_dbs(num_docs, clients, offset):
    fake = Faker()
    pipes = [c.pipeline() for c in clients]
    for i in tqdm(range(num_docs)):
        for pipe in pipes:
            text = fake.sentence().encode("utf-8")
            pipe.hset(f"doc:{i+offset}", mapping={'t'.encode("utf-8"): text})
            if (i + 1) % 10000 == 0:  # Execute pipeline every 1000 commands
                pipe.execute()
        for pipe in pipes:
            pipe.execute()

Sanity Check: should see Max collector result processor in the branch db

In [ ]:
# debug
branch_redis_client.execute_command('ft.profile idx aggregate query * TIMEOUT 0 ')

230503:M 09 Apr 2025 14:49:09.546 # <search> count in the end of the loop is 665856
230503:M 09 Apr 2025 14:49:09.850 # <search> rp_max->index in EOF of rpMax_Yield is 665856


[[665856],
 [b'Shards',
  [[b'Total profile time',
    b'916',
    b'Parsing time',
    b'0',
    b'Pipeline creation time',
    b'0',
    b'Total GIL time',
    b'916.81365',
    b'Warning',
    b'None',
    b'Iterators profile',
    [b'Type', b'WILDCARD', b'Time', b'133', b'Counter', 665856],
    b'Result processors profile',
    [[b'Type', b'Index', b'Time', b'425', b'Counter', 665856],
     [b'Type', b'Max collector', b'Time', b'322', b'Counter', 665856]]]],
  b'Coordinator',
  []]]

In [ ]:
#debug
master_redis_client.execute_command('ft.profile idx aggregate query * TIMEOUT 0 LIMIT 0 0')

[[665857],
 [b'Shards',
  [[b'Total profile time',
    b'576',
    b'Parsing time',
    b'0',
    b'Pipeline creation time',
    b'0',
    b'Total GIL time',
    b'576.506575',
    b'Warning',
    b'None',
    b'Iterators profile',
    [b'Type', b'WILDCARD', b'Time', b'134', b'Counter', 665857],
    b'Result processors profile',
    [[b'Type', b'Index', b'Time', b'432', b'Counter', 665857],
     [b'Type', b'Counter', b'Time', b'144', b'Counter', 1]]]],
  b'Coordinator',
  []]]

In [ ]:
# import threading
import timeit
from pprint import pprint
import json


#query2 : FT.AGGREGATE idx * TIMEOUT 0 LIMIT 0 0 ' and search-max-aggregate-results = fixed v results in v results with no serialization, check this again varying dbsize

# Parameters
query = 'FT.AGGREGATE idx * TIMEOUT 0'
total_docs = 5000001
step = 300000
num_docs = 0
results = {"master": {}, "branch": {}}

def timed_query(client, label, num_docs):
    t = timeit.timeit('client.execute_command(query)', globals={'client': client, 'query': query}, number=1)
    results[label][num_docs] = t

while num_docs < total_docs:
    # Populate DBs synchronously (assuming you want to do this before each measurement)
    populate_dbs(step, [master_redis_client, branch_redis_client], num_docs)
    num_docs += step

    timed_query(branch_redis_client, "branch", num_docs)
    timed_query(master_redis_client, "master", num_docs)


    with open(f"results_{num_docs}_json", "w") as f:
        json.dump(results, f, indent=4)

    # Print results
    pprint(results)

with open("final_results.json", "w") as f:
    json.dump(results, f, indent=4)


  4%|▎         | 10950/300000 [00:02<01:15, 3828.26it/s]

232714:M 09 Apr 2025 15:00:29.977 * 10000 changes in 60 seconds. Saving...
232714:M 09 Apr 2025 15:00:29.978 * Background saving started by pid 232740
232740:C 09 Apr 2025 15:00:29.993 * DB saved on disk
232740:C 09 Apr 2025 15:00:29.994 * Fork CoW for RDB: current 2 MB, peak 2 MB, average 1 MB
232714:M 09 Apr 2025 15:00:30.078 * Background saving terminated with success
232724:M 09 Apr 2025 15:00:30.078 * 10000 changes in 60 seconds. Saving...
232724:M 09 Apr 2025 15:00:30.079 * Background saving started by pid 232741
232741:C 09 Apr 2025 15:00:30.094 * DB saved on disk
232741:C 09 Apr 2025 15:00:30.094 * Fork CoW for RDB: current 2 MB, peak 2 MB, average 1 MB


  4%|▍         | 11721/300000 [00:03<01:17, 3730.62it/s]

232724:M 09 Apr 2025 15:00:30.179 * Background saving terminated with success


 85%|████████▌ | 255006/300000 [01:03<00:13, 3382.78it/s]

...

 69%|██████▉   | 206986/300000 [00:52<00:21, 4378.30it/s]

232873:C 09 Apr 2025 15:21:55.966 * DB saved on disk
232873:C 09 Apr 2025 15:21:56.042 * Fork CoW for RDB: current 58 MB, peak 58 MB, average 30 MB
232724:M 09 Apr 2025 15:21:56.164 * Background saving terminated with success


100%|██████████| 300000/300000 [01:14<00:00, 4023.65it/s]


232724:M 09 Apr 2025 15:22:18.471 # <search> count in the end of the loop is 5100000
232724:M 09 Apr 2025 15:22:18.729 # <search> rp_max->index in EOF of rpMax_Yield is 5100000
{'branch': {300000: 0.0377202300587669,
            600000: 0.07759436697233468,
            900000: 0.11062348599079996,
            1200000: 0.16955550701823086,
            1500000: 0.19398569106124341,
            1800000: 0.22777571703772992,
            2100000: 0.2880714940838516,
            2400000: 0.3292395389871672,
            2700000: 0.367517112987116,
            3000000: 0.4017737649846822,
            3300000: 0.44577393494546413,
            3600000: 0.48042827495373785,
            3900000: 0.5195964860031381,
            4200000: 0.6048267451114953,
            4500000: 0.650207509053871,
            4800000: 0.6935783950611949,
            5100000: 0.7234422769397497},
 'master': {300000: 0.021290407981723547,
            600000: 0.0453962089959532,
            900000: 0.06466361007187515,


232714:M 09 Apr 2025 15:22:56.032 * 10000 changes in 60 seconds. Saving...
232714:M 09 Apr 2025 15:22:56.100 * Background saving started by pid 232876
232724:M 09 Apr 2025 15:22:57.084 * 10000 changes in 60 seconds. Saving...
232724:M 09 Apr 2025 15:22:57.184 * Background saving started by pid 232877
232876:C 09 Apr 2025 15:23:03.143 * DB saved on disk
232876:C 09 Apr 2025 15:23:03.198 * Fork CoW for RDB: current 2 MB, peak 2 MB, average 1 MB
232714:M 09 Apr 2025 15:23:03.323 * Background saving terminated with success
232877:C 09 Apr 2025 15:23:04.157 * DB saved on disk
232877:C 09 Apr 2025 15:23:04.238 * Fork CoW for RDB: current 2 MB, peak 2 MB, average 1 MB
232724:M 09 Apr 2025 15:23:04.408 * Background saving terminated with success


Data setup for plot

In [ ]:
import matplotlib.pyplot as plt

docs = list((results["master"]).keys())

master_values = list((results["master"]).values())
branch_values = list((results["branch"]).values())

ratios = [(b - m) / m * 100 for m, b in zip(master_values, branch_values)]


Plot the results

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FuncFormatter

# First plot (bar chart)
plt.figure(figsize=(12, 6))
plt.bar(docs, master_values, width=80000, label='Without Normalization', align='center', alpha=0.7)
plt.bar(docs, branch_values, width=80000, label='With Normalization', align='edge', alpha=0.7)
plt.xlabel("Number of Docs (Thousands)")
plt.ylabel("Values (Seconds)")  # Assuming 'Values' are in seconds
plt.title("With vs Without Normalization")
plt.legend()
plt.grid(True)

# Adjusting x-axis ticks (horizontal axis in 1e3)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True, prune='lower'))  # Adjust x-axis to show integers
plt.gca().set_xticklabels([f'{x / 1000:.0f}' for x in plt.gca().get_xticks()])  # Scale to thousands

# Adjusting y-axis ticks (vertical axis in seconds)
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x:.2f} s'))  # Format y-axis as seconds

plt.show()

# Second plot (line chart)
plt.figure(figsize=(12, 6))
plt.plot(docs, ratios, marker='o', color='b', label="Δ/ Without Normalization (%)")
plt.xlabel("Number of Docs")
plt.ylabel("Percentage Difference (%)")
plt.title("Percentage Difference Between With vs Without Normalization")
plt.legend()
plt.grid(True)

# Adjusting x-axis ticks for the second plot (horizontal axis in 1e3)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True, prune='lower'))  # Adjust x-axis to show integers
plt.gca().set_xticklabels([f'{x / 1000:.0f}' for x in plt.gca().get_xticks()])  # Scale to thousands

# Adjusting y-axis ticks for the second plot (vertical axis in seconds)
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x:.2f} s'))  # Format y-axis as seconds

plt.show()


NameError: name 'docs' is not defined

<Figure size 1200x600 with 0 Axes>

In [ ]:
import timeit
from pprint import pprint
import json

# Parameters
#two similar queries except sorter result processor in the end of the proccessing
query_sorted = 'FT.AGGREGATE idx * SORTBY 2 @t ASC ADDSCORES TIMEOUT 0'
query_no_sorted= 'FT.AGGREGATE idx * ADDSCORES LOAD 1 @t  TIMEOUT 0 '
total_docs = 5000001
step = 300000
num_docs = 0
results = {"with_sorter": {}, "without_sorter": {}}

def timed_query(client, label, query, num_docs):
    t = timeit.timeit('client.execute_command(query)', globals={'client': client, 'query': query}, number=1)
    results[label][num_docs] = t

redis_client = master_redis_client

while num_docs < total_docs:
    # Populate DB (assuming this is a custom function you have)
    populate_dbs(step, [redis_client], num_docs)
    num_docs += step

    timed_query(redis_client, "with_sorter", query_sorted, num_docs)
    timed_query(redis_client, "without_sorter", query_no_sorted, num_docs)

    with open(f"results_sorting_{num_docs}_json", "w") as f:
        json.dump(results, f, indent=4)

    # Print intermediate results
    pprint(results)

with open("final_results_sorting.json", "w") as f:
    json.dump(results, f, indent=4)

100%|██████████| 300000/300000 [00:26<00:00, 11534.67it/s]


{'with_sorter': {300000: 0.18964799505192786},
 'without_sorter': {300000: 0.23805118899326771}}


 77%|███████▋  | 232282/300000 [00:20<00:06, 9932.27it/s] 

233280:M 09 Apr 2025 16:13:23.020 * 10000 changes in 60 seconds. Saving...
233280:M 09 Apr 2025 16:13:23.063 * Background saving started by pid 233520


 81%|████████  | 242297/300000 [00:20<00:05, 11070.47it/s]

...

 78%|███████▊  | 235458/300000 [00:20<00:05, 11585.09it/s]

233280:M 09 Apr 2025 16:20:59.463 * Background saving terminated with success


100%|██████████| 300000/300000 [00:26<00:00, 11475.16it/s]


{'with_sorter': {300000: 0.18964799505192786,
                 600000: 0.3917022739769891,
                 900000: 0.566300384933129,
                 1200000: 0.7473482619971037,
                 1500000: 0.9958648140309379,
                 1800000: 1.1941409799037501,
                 2100000: 1.88969486404676,
                 2400000: 1.5827165950322524,
                 2700000: 1.869545312016271,
                 3000000: 2.068169769947417,
                 3300000: 2.412668626056984,
                 3600000: 2.6133987059583887,
                 3900000: 2.945143042015843,
                 4200000: 4.077396895037964,
                 4500000: 3.27404904703144,
                 4800000: 3.462774492101744,
                 5100000: 3.843257978092879},
 'without_sorter': {300000: 0.23805118899326771,
                    600000: 0.45434289099648595,
                    900000: 0.7106432620203122,
                    1200000: 0.9378467430360615,
                    1500000: 1.21182

233280:M 09 Apr 2025 16:22:00.095 * 10000 changes in 60 seconds. Saving...
233280:M 09 Apr 2025 16:22:00.269 * Background saving started by pid 233539
233539:C 09 Apr 2025 16:22:07.380 * DB saved on disk
233539:C 09 Apr 2025 16:22:07.522 * Fork CoW for RDB: current 2 MB, peak 2 MB, average 2 MB
233280:M 09 Apr 2025 16:22:07.793 * Background saving terminated with success
